# ML model – klasifikace kategorie článků

Data: vlastní crawl z idnes.cz, novinky.cz, aktualne.cz, irozhlas.cz  
Postup: načtení → předzpracování → trénování → vyhodnocení

## 1. Načtení dat

Cesta k datům: buď `../crawl/articles_part1.jsonl`, nebo po nahrání na Colab disk.

In [ ]:
import json

DATA_PATH = "../crawl/articles_part1.jsonl"  # lokálně; na Colabu změň na cestu k souboru
MAX_ROWS = 5000  # pro rychlý běh; pro finální model můžeš zvýšit nebo použít víc

rows = []
with open(DATA_PATH, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= MAX_ROWS:
            break
        line = line.strip()
        if not line:
            continue
        try:
            rows.append(json.loads(line))
        except json.JSONDecodeError:
            continue

print(f"Načteno záznamů: {len(rows)}")

## 2. Předzpracování (čištění, příprava příznaků)

Odstranění řádků bez obsahu/kategorie, sjednocení kódování, vytvoření vstupů pro model.

In [ ]:
import pandas as pd

df = pd.DataFrame(rows)

# Čištění: jen řádky s obsahu a kategorii
df = df.dropna(subset=["obsah", "kategorie"])
df = df[df["obsah"].str.len() >= 100]
df = df[df["kategorie"].astype(str).str.len() >= 1]

# Sloupec pro text (nadpis + obsah)
df["text"] = (df["nadpis"].fillna("") + " " + df["obsah"]).str.strip()
df = df[df["text"].str.len() >= 150]

# Odstranění duplicit podle URL
df = df.drop_duplicates(subset=["url"])

print(f"Po čištění: {len(df)} záznamů")
print(df["kategorie"].value_counts().head(10))

In [ ]:
# Škálování/transformace: pro text použijeme TF-IDF (knihovna)
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

# Cíl: kategorie
le = LabelEncoder()
y = le.fit_transform(df["kategorie"])

X_train, X_test, y_train, y_test = train_test_split(
    df["text"], y, test_size=0.2, random_state=42, stratify=y
)

# TF-IDF na trénovacích datech, pak transform na test
vec = TfidfVectorizer(max_features=5000, min_df=2, max_df=0.95, ngram_range=(1, 2))
X_train_vec = vec.fit_transform(X_train)
X_test_vec = vec.transform(X_test)

print("Rozměry: train", X_train_vec.shape, ", test", X_test_vec.shape)

## 3. Trénování modelu

Klasifikace: např. logistická regrese nebo jednoduchý klasifikátor.

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=500, random_state=42)
model.fit(X_train_vec, y_train)

## 4. Vyhodnocení

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_pred = model.predict(X_test_vec)
acc = accuracy_score(y_test, y_pred)

print("Accuracy:", round(acc, 4))
print("\nClassification report (top třídy):")
print(classification_report(y_test, y_pred, zero_division=0))
print("Confusion matrix (první řádek/sloupec):")
print(confusion_matrix(y_test, y_pred)[:5, :5])

## 5. Export modelu (pro aplikaci)

Uložení modelu, vektorizéru a label encoderu pro použití v predict.py.

In [ ]:
import joblib

joblib.dump({"model": model, "vectorizer": vec, "label_encoder": le}, "model.joblib")
print("Model uložen do model.joblib")